## Kemal Demirel: 191104091

## Kadir Çağan Eren: 

In [ ]:
import glob
import pathlib
import cv2
import numpy as np 
import pandas as pd 
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt

print (cv2.__version__)


## Read all dataset with images

In [20]:

def traverse_directory(path):
    length = 0
    entered = False
    dataset = []
    for dirpath, dirnames, filenames in os.walk(path):
        if not entered:
            length = len(dirnames)
            entered = True    
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            if(file_path.endswith(".jpg")):
                img = cv2.imread(file_path)
                dataset.append(img)
    labels = []
    for i in range(0,length):
        labels.append(i)
    return dataset, labels


In [21]:
images, labels = traverse_directory(os.getcwd()+"/dataset/train")

Number of images

In [22]:
print(len(images))

2779


Number of fruits and vegetables

In [23]:
print(len(labels))

36


## Define the feature extraction functions

In [24]:
# Define the color histogram feature extractor function
def color_histogram(image):
    hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

In [31]:
# Define the texture analysis feature extractor function
def texture_analysis(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = cv2.LBP_create()
    lbp_img = lbp.compute(gray)
    hist = cv2.calcHist([lbp_img], [0], None, [256], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

In [32]:
dataset = []
for image in images:
    image = cv2.resize(image, (100, 100))
    feature = np.concatenate((color_histogram(image), texture_analysis(image)))
    dataset.append(feature)

ValueError: setting an array element with a sequence.

## Define the feature extraction pipeline

In [ ]:

def extract_features(img_path):
    # Load the image
    img = cv2.imread(img_path)

    # Extract the features
    color_hist = color_histogram(img)
    texture_hist = texture_analysis(img)
    edge_hist = edge_detection(img)

    # Concatenate the features
    features = np.concatenate((color_hist, texture_hist, edge_hist))

    return features

### Extract the features from each fruit image

In [ ]:
features = []
i = 0
for img_path in dataset:
    feature = extract_features(img_path)
    features.append(feature)
    print(i)
    i = i + 1

## Split the dataset into training and testing sets


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

## Train the classifier

In [ ]:
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

## Evaluate the classifier

In [ ]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')